In [125]:
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from joblib import dump
import seaborn as sns

In [138]:
dat = pd.read_csv("FeatureExtraction/analysis_result.csv")
#dat.drop(['b', 'g', 'r', 'edge_entropy'], axis=1, inplace=True)
print(dat.head(1))
#dat = dat[dat.edge_diff > 0]
sns.boxplot(x=dat['edge_diff'])
dat = dat.drop_duplicates()

    entropy    variance    dct_bias  edge_density       cc_b       cc_g  \
0  6.700422  835.759799  101.439431       0.20727  43.535091  41.478838   

        cc_r  edge_noise  edge_entropy  is_adv  
0  14.986072   -5.769231      0.021464       0  


KeyError: 'edge_diff'

In [139]:
len(dat)

2555

In [140]:
dat['adv'].unique()

KeyError: 'adv'

In [141]:
x = dat.iloc[:, :-1].values
x = normalize(x, norm='l2')
y = dat.iloc[:,-1].values

In [142]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
#y_train = y_train.reshape([len(y_train), 1])
#y_test = y_test.reshape([len(y_test), 1])

In [143]:
#Remove scaler

sc = StandardScaler()
#x_train = sc.fit_transform(x_train)
#x_test = sc.transform(x_test)
dump(sc, 'Scaler.bin', compress=True)

['Scaler.bin']

In [144]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [145]:
keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

clf = Sequential([
    Dense(512, kernel_initializer='uniform', input_dim=9, activation='elu'),
    Dense(1024, kernel_initializer='uniform', activation='elu'),
    Dropout(0.2),
    Dense(1024, kernel_initializer='uniform', activation='elu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [146]:
clf.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 512)               5120      
_________________________________________________________________
dense_37 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_38 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 1025      
Total params: 1,581,057
Trainable params: 1,581,057
Non-trainable params: 0
_________________________________________________________________


In [147]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [148]:
clf.fit(x_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
56/56 [==============================] - 2s 12ms/step - loss: 0.6187 - accuracy: 0.6212

In [149]:
score = clf.evaluate(x_test, y_test, verbose=0)

print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[0.00022613578767050058, 1.0]


In [107]:
from keras.models import load_model
clf.save('AEGuard.keras')

In [108]:
x_predict = [
    [7.3378496842073835,2005.3647338785904,4.504145408163265,32.902195839142415,33.678372772158376,33.41943138869921,75.08849557522123,0.2649477141230245],
    [7.347049941119418,2043.8005363495167,3.0113998724489797,32.896276164881336,33.6823649646088,33.42135887050986,31.436135009927202,0.19496323402345217]
    #[7.109481332039627,1181.620735253842,4.512117346938775,26.801098135999048,37.14988650707845,36.0490153569225,88.47173144876325,0.2652988602177302],
    #[7.145613536455951,1342.1639741820782,3.559470663265306,26.234164188403792,37.466724054725475,36.29911175687073,58.006718924972,0.2217159457752795],
]
#x_predict = sc.transform(x_predict)
y_predict = clf.predict(x_predict)
print(y_predict)

print("Predicted: ", y_predict[0][0], "- adversarial(> 0.5)")
print("Predicted: ", y_predict[1][0], "- original(< 0.5)")

ValueError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute